<a href="https://colab.research.google.com/github/shivanssharma/Deep-Learning-Practise-/blob/main/24040208006_MDSC_302(P)_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from torchvision import datasets, transforms
data_path = '/content/p1ch7/'



In [2]:
# Load the CIFAR10 training dataset
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

100%|██████████| 170M/170M [00:13<00:00, 12.3MB/s]


In [3]:
# Load the CIFAR10 validation dataset
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468), # Normalize the image tensors with mean and standard deviation
                             (0.2470, 0.2435, 0.2616))
    ]))

In [4]:
label_map = {0: 0, 2: 1}
# Define the names for the new classes
class_names = ['airplane', 'bird']

cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Create a DataLoader for the training dataset
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)

    # Define the forward pass
    def forward(self, x):
        # Apply first convolution, ReLU activation, and 2x2 max pooling
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        # Apply second convolution, ReLU activation, and 2x2 max pooling
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        # Flatten the output for the fully connected layers
        out = out.view(-1, 8 * 8 * 8)
        # Apply first fully connected layer and tanh activation
        out = torch.tanh(self.fc1(out))
        # Apply second fully connected layer (output layer)
        out = self.fc2(out)
        return out

# Instantiate the network model
model = Net()

# Define the learning rate
learning_rate = 1e-2

# Define the optimizer (Stochastic Gradient Descent)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Define the loss function (Cross-Entropy Loss for classification)
loss_fn = nn.CrossEntropyLoss()

# Define the number of training epochs
n_epochs = 100

# Training loop
for epoch in range(n_epochs):
    # Iterate over batches in the training data
    for imgs, labels in train_loader:
        # Get model outputs for the current batch
        outputs = model(imgs)
        # Calculate the loss
        loss = loss_fn(outputs, labels)

        # Zero the gradients, perform backpropagation, and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss after each epoch
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

/tmp/ipython-input-957594607.py:63: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))


Epoch: 0, Loss: 0.510457
Epoch: 1, Loss: 0.432535
Epoch: 2, Loss: 1.004270
Epoch: 3, Loss: 0.337026
Epoch: 4, Loss: 0.239355
Epoch: 5, Loss: 0.321709
Epoch: 6, Loss: 0.229540
Epoch: 7, Loss: 0.257699
Epoch: 8, Loss: 0.275317
Epoch: 9, Loss: 0.177018
Epoch: 10, Loss: 0.406675
Epoch: 11, Loss: 0.187603
Epoch: 12, Loss: 0.327529
Epoch: 13, Loss: 0.679993
Epoch: 14, Loss: 0.298198
Epoch: 15, Loss: 0.469005
Epoch: 16, Loss: 0.326155
Epoch: 17, Loss: 0.278744
Epoch: 18, Loss: 0.252449
Epoch: 19, Loss: 0.210714
Epoch: 20, Loss: 0.354748
Epoch: 21, Loss: 0.070826
Epoch: 22, Loss: 0.315485
Epoch: 23, Loss: 0.113934
Epoch: 24, Loss: 0.449574
Epoch: 25, Loss: 0.273074
Epoch: 26, Loss: 0.242370
Epoch: 27, Loss: 0.147039
Epoch: 28, Loss: 0.097489
Epoch: 29, Loss: 0.050351
Epoch: 30, Loss: 0.140751
Epoch: 31, Loss: 0.141717
Epoch: 32, Loss: 0.375215
Epoch: 33, Loss: 0.493140
Epoch: 34, Loss: 0.383294
Epoch: 35, Loss: 0.193899
Epoch: 36, Loss: 0.114928
Epoch: 37, Loss: 0.240468
Epoch: 38, Loss: 0.462

In [6]:
# Create a DataLoader for the training dataset, ensuring shuffle is False for consistent evaluation
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

# Disable gradient calculation for evaluation
with torch.no_grad():
    # Iterate over the training data
    for imgs, labels in train_loader:
        # Get model predictions. Pass the images directly to the model.
        outputs = model(imgs)
        # Get the predicted class by finding the index of the maximum output value
        _, predicted = torch.max(outputs, dim=1)
        # Update the total number of samples
        total += labels.shape[0]
        # Update the number of correct predictions
        correct += int((predicted == labels).sum())

# Print the accuracy on the training set
print("Accuracy: %f" % (correct / total))

Accuracy: 0.964300


In [7]:
# Create a DataLoader for the validation dataset, ensuring shuffle is False for consistent evaluation
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

# Disable gradient calculation for evaluation
with torch.no_grad():
    # Iterate over the validation data
    for imgs, labels in val_loader:
        # Get model predictions. Pass the images directly to the model.
        outputs = model(imgs)
        # Get the predicted class by finding the index of the maximum output value
        _, predicted = torch.max(outputs, dim=1)
        # Update the total number of samples
        total += labels.shape[0]
        # Update the number of correct predictions
        correct += int((predicted == labels).sum())

# Print the accuracy on the validation set
print("Accuracy: %f" % (correct / total))

Accuracy: 0.880000
